In [169]:
import pandas as pd
import numpy as np
import json
import scipy
import getpass
import socket
import random
import math

In [46]:
TEAM_KEY = getpass.getpass("TEAM_KEY: ")

TEAM_KEY: ········


In [157]:
def bytelen(obj):  
    if type(obj) == str:
        obj = obj.encode()
    size = len(bytes(obj))
    return bytes((size % (256 ** i) // 256**(i-1) for i in range(1, 5)))

def byte_count(x):
    x = list(x)
    return sum((x[i] * 256**i for i in range(len(x))))

def connect_to_server(url, port):
    sock = socket.socket()
    sock.connect((url, port))
    return sock

def send_bytes(sock, data):
    if type(data) == str:
        data = data.encode()
    data = bytes(data)
    return sock.send(bytelen(data) + data)

def recv_data(sock):
    l = sock.recv(4)
    l = byte_count(l)
    res = []
    cnt = 0
    #print(l)
    while cnt < l:
        tmp = list(sock.recv(1000000 if l - cnt > 1000000 else l - cnt))
        cnt += len(tmp)
        res.extend(tmp)
    #print(cnt)
    return bytes(res)


def dict_to_bytes(data):
    return json.dumps(data)

def send_result(sock, x, y, ready):
    data = {"x": x, "y":y, "ready": ready}
    return send_bytes(sock, dict_to_bytes(data))

def get_map(sock):
    return json.loads(recv_data(sock))

In [ ]:
hello_content = json.dumps({'team': TEAM_KEY, 'task': 1}).encode()
TIME_OUT = 10

In [179]:
#Communication loop
sock = connect_to_server("besthack19.sytes.net", 4242)
sock.settimeout(TIME_OUT)
send_bytes(sock, hello_content)
landMap = get_map(sock)
mapLen = int(math.sqrt(len(landMap)))
if landMap.get('error'):
    print("ERROR: ", landMap["error"])
    exit()
landMap = landMap["map"]
while (True):
    toJson = recv_data(sock)
    data = json.loads(toJson.decode())
    if data.get('error'):
        print("ERROR: ", data["error"])
        break
    if data.get('scores'):
        print("SCORE ",data['scores'])
        break
    send_result(sock, data['data']['x'] + 100, data['data']['y'] + 100, 1)

gaierror: [Errno -2] Name or service not known